<a href="https://colab.research.google.com/github/jvrscak/UUIProjekt/blob/main/drugi_zadatak.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

DRUGI ZADATAK - simulacija prometa

In [1]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML
import numpy as np
import random

In [2]:
N = 10          # duljina cesta
M = 12          # broj stanica kružnog toka
TIME_STEPS = 200
MAX_SPEED = 2
ULAZNA_VJEROVATNOST = 0.25  # vjerojatnost pojavljivanja novog vozila po koraku
MIN_SPAWN_INTERVAL = 2      # minimalni interval između pojavljivanja vozila

SEMAFOR_CYCLE = 20          # ciklus semafora (koraci)
SEMAFOR_GREEN_TIME = 12     # vrijeme zelenog svjetla (koraci)

ROUNDABOUT_RADIUS = 3
theta = np.linspace(0, 2*np.pi, M, endpoint=False)
round_cells = [(ROUNDABOUT_RADIUS*np.cos(a), ROUNDABOUT_RADIUS*np.sin(a)) for a in theta]

In [3]:
# Ceste (ulazi i izlazi)
cestaA = {i: None for i in range(N)}
cestaB = {i: None for i in range(N)}
cestaC = {i: None for i in range(N)}
cestaD = {i: None for i in range(N)}

# Ceste u suprotnom smjeru (izlazi iz kružnog toka)
cestaA_reverse = {i: None for i in range(N)}
cestaB_reverse = {i: None for i in range(N)}
cestaC_reverse = {i: None for i in range(N)}
cestaD_reverse = {i: None for i in range(N)}

# Automobili
road_cars = {}
round_cars = {}

# Semafori
semafori = {
    'A': (1, 0),  # (faza, timer)
    'B': (0, 0),
    'C': (0, 0),
    'D': (0, 0),
    'A_reverse': (1, 0),
    'B_reverse': (0, 0),
    'C_reverse': (0, 0),
    'D_reverse': (0, 0)
}

In [4]:
# Brojač za ID vozila
next_car_id = 0

# Zadnje vrijeme pojavljivanja vozila po ulazu
last_spawn_time = {'A': -MIN_SPAWN_INTERVAL, 'B': -MIN_SPAWN_INTERVAL,
                   'C': -MIN_SPAWN_INTERVAL, 'D': -MIN_SPAWN_INTERVAL,
                   'A_reverse': -MIN_SPAWN_INTERVAL, 'B_reverse': -MIN_SPAWN_INTERVAL,
                   'C_reverse': -MIN_SPAWN_INTERVAL, 'D_reverse': -MIN_SPAWN_INTERVAL}

In [5]:
def next_round_index(idx, speed):
    """Pomak u kružnom toku"""
    return (idx + speed) % M

def distance_next_round(round_cars, idx):
    """Udaljenost do sljedećeg vozila u kružnom toku"""
    positions = [pos for pos, _, _, _ in round_cars.values()]
    gap = 0
    cur = idx
    while True:
        cur = (cur + 1) % M
        gap += 1
        if cur in positions or gap > M:
            break
    return gap - 1

def distance_next_road(road_cars, pos, cesta_name):
    """Udaljenost do sljedećeg vozila na cesti"""
    # Za normalne ceste (A,B,C,D): vozila idu od N-1 (van) prema 0 (kružni tok) - ULAZ
    # Za reverse ceste (A_reverse,B_reverse,C_reverse,D_reverse): vozila idu od 0 (kružni tok) prema N-1 (van) - IZLAZ
    is_reverse = '_reverse' in cesta_name

    if is_reverse:
        # Na reverse cesti, vozila idu naprijed (od 0 prema N-1) - izlaz iz kružnog toka
        occupied = [p for c, p, _, _, _ in road_cars.values() if c == cesta_name and p > pos]
        if occupied:
            return min(occupied) - pos - 1
        else:
            return N - pos - 1  # Do kraja ceste (pozicija N-1)
    else:
        # Na normalnoj cesti, vozila idu unazad (od N-1 prema 0) - ulaz u kružni tok
        occupied = [p for c, p, _, _, _ in road_cars.values() if c == cesta_name and p < pos]
        if occupied:
            return pos - max(occupied) - 1
        else:
            return pos  # Do kraja ceste (pozicija 0 - ulaz u kružni tok)

In [6]:
def get_random_exit(entry):
    """Odaberi nasumičan izlaz za vozilo koje ulazi s određenog ulaza"""
    # Ulazi: A=0, B=3, C=6, D=9, A_reverse=0, B_reverse=3, C_reverse=6, D_reverse=9
    # Izlazi: A_reverse=0, B_reverse=3, C_reverse=6, D_reverse=9 (za vozila koja ulaze s A,B,C,D)
    #         A=0, B=3, C=6, D=9 (za vozila koja ulaze s A_reverse,B_reverse,C_reverse,D_reverse)

    entry_map = {'A': 0, 'B': 3, 'C': 6, 'D': 9,
                 'A_reverse': 0, 'B_reverse': 3, 'C_reverse': 6, 'D_reverse': 9}
    entry_idx = entry_map[entry]

    # Ne može izaći na istom ulazu gdje je ušao
    possible_exits = [0, 3, 6, 9]
    possible_exits.remove(entry_idx)

    return random.choice(possible_exits)

def update_semafori(t):
    """Ažuriraj stanje semafora"""
    global semafori

    for ulaz in semafori:
        phase, timer = semafori[ulaz]
        timer += 1

        if phase == 1:  # zeleno
            if timer >= SEMAFOR_GREEN_TIME:
                phase = 0  # prebaci na crveno
                timer = 0
        else:  # crveno
            if timer >= (SEMAFOR_CYCLE - SEMAFOR_GREEN_TIME):
                phase = 1  # prebaci na zeleno
                timer = 0

        semafori[ulaz] = (phase, timer)

def spawn_new_car(t):
    """Generiraj nova vozila na ulazima (samo na normalnim cestama koje ulaze u kružni tok)"""
    global next_car_id, road_cars, last_spawn_time

    # Vozila se pojavljuju samo na normalnim cestama (A, B, C, D) koje ulaze u kružni tok
    # Reverse ceste (A_reverse, B_reverse, C_reverse, D_reverse) su samo za izlazak vozila iz kružnog toka
    ulazi = ['A', 'B', 'C', 'D']

    for ulaz in ulazi:
        # Provjeri je li prošao minimalni interval
        if t - last_spawn_time[ulaz] < MIN_SPAWN_INTERVAL:
            continue

        # Provjeri je li semafor zelen
        if semafori[ulaz][0] == 0:  # crveno
            continue

        # Provjeri je li prva pozicija slobodna
        # Za normalne ceste: prva pozicija je N-1 (počinje s vanjske strane, ide prema kružnom toku)
        start_pos = N-1

        if any(c == ulaz and p == start_pos for c, p, _, _, _ in road_cars.values()):
            continue

        # Nasumično odluči hoće li se pojaviti novo vozilo
        if random.random() < ULAZNA_VJEROVATNOST:
            target_exit = get_random_exit(ulaz)
            road_cars[next_car_id] = (ulaz, start_pos, random.randint(1, MAX_SPEED), target_exit, ulaz)
            next_car_id += 1
            last_spawn_time[ulaz] = t

In [ ]:
def update_system(t):
    global road_cars, round_cars

    # Ažuriraj semafore
    update_semafori(t)

    # Generiraj nova vozila
    spawn_new_car(t)

    # --- Ceste ---
    new_road_cars = {}
    for car_id, (cesta_name, pos, speed, target_exit, entry_road) in road_cars.items():
        is_reverse = '_reverse' in cesta_name

        speed = min(speed+1, MAX_SPEED)
        gap = distance_next_road(road_cars, pos, cesta_name)
        speed = min(speed, gap)

        # Na normalnim cestama (A,B,C,D): vozila idu unazad (od N-1 prema 0) - ulaz u kružni tok
        # Na reverse cestama (A_reverse,B_reverse,C_reverse,D_reverse): vozila idu naprijed (od 0 prema N-1) - izlaz iz kružnog toka
        if is_reverse:
            new_pos = pos + speed  # Reverse ceste: naprijed (0 -> N-1)
        else:
            new_pos = pos - speed  # Normalne ceste: unazad (N-1 -> 0)

        # ulazak u kružni tok ako dosegne kraj
        ulaz_map = {'A': 0, 'B': 3, 'C': 6, 'D': 9,
                     'A_reverse': 0, 'B_reverse': 3, 'C_reverse': 6, 'D_reverse': 9}

        # Za normalne ceste: ulazak kada je vozilo na poziciji 0 ili je prošlo (new_pos <= 0)
        # Za reverse ceste: vozila ne ulaze u kružni tok (već izlaze iz njega)
        if not is_reverse:
            # Provjeri je li vozilo stiglo do kraja ceste (pozicija 0)
            if pos == 0 or new_pos <= 0:
                idx = ulaz_map[cesta_name]
                # Provjeri je li pozicija u kružnom toku slobodna
                if all(idx != r for r, _, _, _ in round_cars.values()):
                    round_cars[car_id] = (idx, speed, target_exit, cesta_name)
                    continue
                else:
                    # Ako je pozicija zauzeta, vozilo se zaustavlja na poziciji 0
                    new_pos = 0
        else:
            # Za reverse ceste: vozila nestaju kada dođu do kraja ceste (pozicija N-1)
            if pos >= N-1 or new_pos >= N-1:
                # Vozilo je došlo do kraja ceste - nestaje
                continue

        # Ograniči poziciju na valjane granice
        if is_reverse:
            new_road_cars[car_id] = (cesta_name, min(new_pos, N-1), speed, target_exit, entry_road)
        else:
            new_road_cars[car_id] = (cesta_name, max(new_pos, 0), speed, target_exit, entry_road)
    road_cars = new_road_cars

    # --- Kružni tok ---
    new_round_cars = {}
    # Mapiranje indeksa na izlaz - vozila koja ulaze s A,B,C,D izlaze na A_reverse,B_reverse,C_reverse,D_reverse
    # Vozila koja ulaze s A_reverse,B_reverse,C_reverse,D_reverse izlaze na A,B,C,D
    # Ali za jednostavnost, koristimo isti exit_map i onda određujemo reverse cestu
    exit_map = {0: 'A', 3: 'B', 6: 'C', 9: 'D'}
    exit_reverse_map = {0: 'A_reverse', 3: 'B_reverse', 6: 'C_reverse', 9: 'D_reverse'}

    for car_id, (idx, speed, target_exit, entry_road) in round_cars.items():
        # Vozila u kružnom toku se pomiču za 1 poziciju po koraku
        # Provjeri je li sljedeća pozicija slobodna
        next_pos = (idx + 1) % M
        if any(next_pos == r for r, _, _, _ in round_cars.values()):
            # Sljedeća pozicija je zauzeta, vozilo se ne pomiče
            new_idx = idx
        else:
            # Sljedeća pozicija je slobodna, vozilo se pomiče za 1 poziciju
            new_idx = next_pos

        # Provjeri je li vozilo stiglo do svog ciljnog izlaza
        # Vozilo može izaći ako je na ciljnom izlazu ili ga je prošlo
        reached_exit = False

        # Provjeri je li vozilo na ciljnom izlazu
        if new_idx == target_exit or idx == target_exit:
            reached_exit = True
        # Provjeri je li vozilo prošlo kroz ciljni izlaz
        elif idx != target_exit:
            # Slučaj 1: Normalan slučaj - vozilo ide prema izlazu (idx < target_exit)
            if idx < target_exit:
                if new_idx >= target_exit:
                    reached_exit = True
            # Slučaj 2: Vozilo je prošlo kroz 0 (idx > target_exit)
            elif idx > target_exit:
                # Vozilo je prošlo kroz 0 i sada je na ili iza ciljnog izlaza
                if new_idx < idx:  # Vozilo je prošlo kroz 0
                    if new_idx >= target_exit:
                        reached_exit = True

        if reached_exit:
            # Odredi na koju cestu vozilo izlazi
            # Ako je vozilo ušlo s A,B,C,D, izlazi na A_reverse,B_reverse,C_reverse,D_reverse
            # Ako je vozilo ušlo s A_reverse,B_reverse,C_reverse,D_reverse, izlazi na A,B,C,D
            is_entry_reverse = '_reverse' in entry_road if entry_road else False

            if is_entry_reverse:
                # Vozilo je ušlo s reverse ceste, izlazi na normalnu cestu
                exit_cesta = exit_map[target_exit]
                start_pos = N-1  # Normalna cesta počinje na poziciji N-1 (van, ide prema kružnom toku)
            else:
                # Vozilo je ušlo s normalne ceste, izlazi na reverse cestu
                exit_cesta = exit_reverse_map[target_exit]
                start_pos = 0  # Reverse cesta počinje na poziciji 0 (kružni tok, ide prema van)

            # Provjeri da je pozicija na cesti slobodna
            # Ako je pozicija slobodna, vozilo izlazi
            if all(not (cn == exit_cesta and p == start_pos) for cn, p, _, _, _ in road_cars.values()):
                road_cars[car_id] = (exit_cesta, start_pos, speed, None, None)  # None jer više nije u kružnom toku
                continue
            # Ako je pozicija zauzeta, vozilo ostaje u kružnom toku i pokušava izaći sljedeći korak
            # Ali ako je vozilo već prošlo kroz izlaz (new_idx > target_exit ili new_idx < idx),
            # možda bi trebalo omogućiti izlazak na sljedećoj poziciji
            # Za sada, vozilo ostaje u kružnom toku i pokušava izaći sljedeći korak

        new_round_cars[car_id] = (new_idx, speed, target_exit, entry_road)
    round_cars = new_round_cars